In [29]:
import pandas as pd
import numpy as np
import random as rnd
import csv
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [6]:
train = pd.read_csv('C:/Users/bodak/Desktop/train.csv')
test = pd.read_csv('C:/Users/bodak/Desktop/test.csv')
combine = [train,test]
y_train = train['Survived']

In [7]:
train.drop(labels="Survived", axis=1, inplace=True)

for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

In [8]:
train = train.drop(['Name'], axis=1)
test = test.drop(['Name'], axis=1)
train = train.drop(['Ticket', 'Cabin'], axis=1)
test = test.drop(['Ticket', 'Cabin'], axis=1)
combine = [train, test]

In [9]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,3,0,22.0,1,0,7.2500,S,1
1,2,1,1,38.0,1,0,71.2833,C,3
2,3,3,1,26.0,0,0,7.9250,S,2
3,4,1,1,35.0,1,0,53.1000,S,3
4,5,3,0,35.0,0,0,8.0500,S,1


In [10]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,892,3,0,34.5,0,0,7.8292,Q,1
1,893,3,1,47.0,1,0,7.0000,S,3
2,894,2,0,62.0,0,0,9.6875,Q,1
3,895,3,0,27.0,0,0,8.6625,S,1
4,896,3,1,22.0,1,1,12.2875,S,3


In [11]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 25), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 25) & (dataset['Age'] <= 32), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 4
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 5

In [12]:
train = train.drop(['Parch', 'SibSp'], axis=1)
test = test.drop(['Parch', 'SibSp'], axis=1)
combine = [train, test]

In [13]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Sex            418 non-null int32
Age            332 non-null float64
Fare           417 non-null float64
Embarked       418 non-null object
Title          418 non-null int64
dtypes: float64(2), int32(1), int64(3), object(1)
memory usage: 21.3+ KB


In [15]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)


In [16]:
test['Fare'].fillna(test['Fare'].dropna().median(), inplace=True)
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

In [17]:
for dataset in combine:    
    dataset.loc[( dataset['Title'] == 1) , 'Age'] = dataset["Age"].fillna(1)
    dataset.loc[( dataset['Title'] == 2) , 'Age'] = dataset["Age"].fillna(1)
    dataset.loc[( dataset['Title'] == 3) , 'Age'] = dataset["Age"].fillna(2)
    dataset.loc[ (dataset['Title'] == 4) , 'Age'] = dataset["Age"].fillna(0)
    dataset.loc[ (dataset['Title'] == 5) , 'Age'] = dataset["Age"].fillna(3)

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
PassengerId    891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null int32
Age            891 non-null float64
Fare           891 non-null int32
Embarked       891 non-null int32
Title          891 non-null int64
dtypes: float64(1), int32(3), int64(3)
memory usage: 38.4 KB


In [19]:
scaler = MinMaxScaler()
train_scale = scaler.fit_transform(train)
test_scale = scaler.transform(test)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(train_scale, y_train, random_state=42)

In [21]:
test.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title
0,892,3,0,3.0,0,2,1
1,893,3,1,3.0,0,0,3
2,894,2,0,4.0,1,2,1
3,895,3,0,2.0,1,0,1
4,896,3,1,1.0,1,0,3


In [22]:
train.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title
0,1,3,0,1.0,0,0,1
1,2,1,1,3.0,3,1,3
2,3,3,1,2.0,1,0,2
3,4,1,1,3.0,3,0,3
4,5,3,0,3.0,1,0,1


In [23]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=20, learning_rate = learning_rate, max_features=2, max_depth = 2, random_state = 241)
    gb.fit(X_train, y_train)
    print(gb.score(X_train, y_train))
    print(gb.score(X_test, y_test))
    print(learning_rate)

0.7919161676646707
0.7847533632286996
0.05
0.8023952095808383
0.7847533632286996
0.1
0.8188622754491018
0.7982062780269058
0.25
0.8338323353293413
0.8116591928251121
0.5
0.8323353293413174
0.8295964125560538
0.75
0.844311377245509
0.820627802690583
1


In [24]:
gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.75, max_features=2, max_depth = 2, random_state = 241)
gb.fit(X_train, y_train)
print(gb.score(X_train, y_train))
print(gb.score(X_test, y_test))
predictions = gb.predict(test_scale)

0.8323353293413174
0.8295964125560538


In [25]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [61]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": predictions })
submission.to_csv('C:/Users/bodak/Desktop/my_submission.csv', index = False)

In [55]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,0
7,899,0
8,900,0
9,901,0
